## Extract features for patients received Heparin. Some tables are obtained by running the official GitHub repo.

In [1]:
#libraries
import numpy as np
import pandas as pd
import psycopg2 #used to connect to our local MIMIC-III database
import collections
# import getpass
from datetime import datetime
import os,sys,re
import pickle
import csv
import math
#import seaborn as sns
# import random
from datetime import timedelta
from pathlib import Path
import importlib
import bisect
import glob
import copy


from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

#%load_ext autotime

{'width': 1024, 'height': 768, 'scroll': True}

In [2]:
dbname = 'mimiciv'
password = '15289943821'
user = 'postgres'
conn = psycopg2.connect(dbname=dbname, password=password,user=user,port=5433)
cur=conn.cursor()




#####   icustay_id (MIMICIII) = stay_id (MIMICIV)

### SQL statement to build lab test table

In [ ]:
'''
DROP TABLE IF EXISTS public.pivoted_lab; CREATE TABLE public.pivoted_lab AS 

SELECT
  MAX(le.subject_id) AS subject_id,
  MAX(le.hadm_id) AS hadm_id,
  MAX(le.charttime) AS charttime,
  MAX(icu.stay_id) as stay_id,
  le.specimen_id, 
  MAX(CASE WHEN le.itemid = 50862 AND le.valuenum <= 10 THEN le.valuenum ELSE NULL END) AS albumin,
  MAX(CASE WHEN le.itemid = 50930 AND le.valuenum <= 10 THEN le.valuenum ELSE NULL END) AS globulin,
  MAX(CASE WHEN le.itemid = 50976 AND le.valuenum <= 20 THEN le.valuenum ELSE NULL END) AS total_protein,
  MAX(CASE WHEN le.itemid = 50868 AND le.valuenum <= 10000 THEN le.valuenum ELSE NULL END) AS aniongap,
  MAX(CASE WHEN le.itemid = 50882 AND le.valuenum <= 10000 THEN le.valuenum ELSE NULL END) AS bicarbonate,
  MAX(CASE WHEN le.itemid = 51006 AND le.valuenum <= 300 THEN le.valuenum ELSE NULL END) AS bun,
  MAX(CASE WHEN le.itemid = 50893 AND le.valuenum <= 10000 THEN le.valuenum ELSE NULL END) AS calcium,
  MAX(CASE WHEN le.itemid = 50902 AND le.valuenum <= 10000 THEN le.valuenum ELSE NULL END) AS chloride,
  MAX(CASE WHEN le.itemid = 50912 AND le.valuenum <= 150 THEN le.valuenum ELSE NULL END) AS creatinine,
  MAX(CASE WHEN le.itemid = 50931 AND le.valuenum <= 10000 THEN le.valuenum ELSE NULL END) AS glucose,
  MAX(CASE WHEN le.itemid = 50983 AND le.valuenum <= 200 THEN le.valuenum ELSE NULL END) AS sodium,
  MAX(CASE WHEN le.itemid = 50971 AND le.valuenum <= 30 THEN le.valuenum ELSE NULL END) AS potassium,
  MAX(CASE WHEN le.itemid = 51237 THEN le.valuenum ELSE NULL END) AS inr,
  MAX(CASE WHEN le.itemid = 51274 THEN le.valuenum ELSE NULL END) AS pt,
  MAX(CASE WHEN le.itemid = 51275 THEN le.valuenum ELSE NULL END) AS ptt,
  MAX(CASE WHEN le.itemid = 51221 THEN le.valuenum ELSE NULL END) AS hematocrit,
  MAX(CASE WHEN le.itemid = 51222 THEN le.valuenum ELSE NULL END) AS hemoglobin,
  MAX(CASE WHEN le.itemid = 51248 THEN le.valuenum ELSE NULL END) AS mch,
  MAX(CASE WHEN le.itemid = 51249 THEN le.valuenum ELSE NULL END) AS mchc,
  MAX(CASE WHEN le.itemid = 51250 THEN le.valuenum ELSE NULL END) AS mcv,
  MAX(CASE WHEN le.itemid = 51265 THEN le.valuenum ELSE NULL END) AS platelet,
  MAX(CASE WHEN le.itemid = 51279 THEN le.valuenum ELSE NULL END) AS rbc,
  MAX(CASE WHEN le.itemid = 51277 THEN le.valuenum ELSE NULL END) AS rdw,
  MAX(CASE WHEN le.itemid = 52159 THEN le.valuenum ELSE NULL END) AS rdwsd,
  MAX(CASE WHEN le.itemid = 51301 THEN le.valuenum ELSE NULL END) AS wbc
  
FROM public.labevents AS le join public.icustays icu on le.subject_id = icu.subject_id 
WHERE
  le.itemid IN (50862,50930,50976,50868 ,50882 ,51006 ,50893 ,50902 ,50912 ,50931,50983,50971 ,51237,51274,51275,51221,51222,
  51248,51249,51250 ,51265,51279 ,51277,52159 ,51301)
  AND NOT valuenum IS NULL
  AND /* lab values cannot be 0 and cannot be negative */ /* .. except anion gap. */ (
    valuenum > 0 OR itemid = 50868
  )
  AND icu.intime <= charttime AND charttime <= icu.outtime
  AND not icu.stay_id is null
GROUP BY
  le.specimen_id

'''

## SQL statement to build patient_info table

In [ ]:
query = '''
DROP TABLE IF EXISTS public.patient_info; CREATE TABLE public.patient_info AS (

with ss as ( select icu.stay_id, info.age, info.is_male, info.is_white, info.is_black, info.is_hispanic, info.is_asian, info.is_other from
icustays icu join (
    select pt.gender, ad.hadm_id, pt.anchor_age as age, case WHEN pt.gender = 'M' then 1 else 0 end as is_male, ad.is_white, ad.is_black,
    ad.is_hispanic, ad.is_asian, ad.is_other from  public.patients pt join (
    select subject_id, hadm_id, race, is_black,is_white,is_hispanic,is_asian, (is_other1+is_other2+is_other3+is_other4-3) as is_other from
(select subject_id, hadm_id, race,
       case WHEN race like '%BLACK%' then 1 else 0 end as is_black,
       case WHEN race like '%WHITE%' then 1 else 0 end as is_white,
       case WHEN race like '%HISPANIC%' then 1 else 0 end as is_hispanic,
       case WHEN race like '%ASIAN%' then 1 else 0 end as is_asian,
       case WHEN race not like '%BLACK%' then 1 else 0 end as is_other1,
       case WHEN race not like '%WHITE%' then 1 else 0 end as is_other2,
       case WHEN race not like '%HISPANIC%' then 1 else 0 end as is_other3,
       case WHEN race not like '%ASIAN%' then 1 else 0 end as is_other4 from admissions) ad 
    ) ad on pt.subject_id = ad.subject_id 
) info on icu.hadm_id = info.hadm_id)

select ss.stay_id,  ss.age, ss.is_male, ss.is_black, ss.is_white,ss.is_hispanic,
 ss.is_ASIAN, ss.is_OTHER,  mv.weight
from ss join ( select distinct stay_id, weight from first_day_weight) mv on ss.stay_id = mv.stay_id where
ss.age is not null and ss.is_male is not null and ss.is_black is not null and ss.is_white is not null and 
ss.is_hispanic is not null and ss.is_ASIAN is not null and ss.is_OTHER is not null and mv.weight is not null)
'''

### Select  cohort

In [3]:
query_cohort = 'select * from patient_info where stay_id in (select distinct stay_id from public.chartevents where itemid = 229375 )'

    
query_pivoted_lab = '''
select * from public.pivoted_lab where stay_id is not null
'''

sql_statement = '''
select * from (
with ss as (
'''\
+ query_cohort + ' ),  pvl as (' \
+ query_pivoted_lab + ') '\
+'''
select pvl.subject_id as subject_id, ss.stay_id as stay_id, 
ss.age as age, ss.is_male as is_male, ss.is_black as is_black,  ss.is_white 
as is_white, ss.is_hispanic as is_hispanic, ss.is_ASIAN as is_ASIAN, ss.is_OTHER as is_OTHER,ss.weight as weight, 
pvl.charttime as pvl_charttime, pvl.aniongap as aniongap, pvl.albumin as albumin, 
pvl.bicarbonate as bicarbonate,  pvl.creatinine as creatinine,pvl.chloride as chloride,
pvl.hematocrit as hematocrit,pvl.hemoglobin as hemoglobin,
pvl.platelet as platelet,pvl.potassium as potassium,pvl.ptt as ptt,pvl.inr as inr,pvl.pt as pt,
pvl.sodium as sodium,pvl.bun as bun,pvl.wbc as wbc 
from ss join  pvl on ss.stay_id=pvl.stay_id
) cohort 

''' 



ppt_df = pd.read_sql_query(sql_statement,conn) 
ppt_df

,subject_id,stay_id,age,is_male,is_black,is_white,is_hispanic,is_asian,is_other,weight,...,hematocrit,hemoglobin,platelet,potassium,ptt,inr,pt,sodium,bun,wbc
0,16175244,36032605,32,1,0,1,0,0,0,82.200000,...,NaN,NaN,NaN,4.2,NaN,NaN,NaN,143.0,30.0,NaN
1,16133115,38070632,57,1,0,1,0,0,0,139.288889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18393915,32951543,65,0,0,1,0,0,0,60.200000,...,NaN,NaN,NaN,4.1,NaN,NaN,NaN,143.0,99.0,NaN
3,13060420,34563409,57,1,0,0,1,0,0,85.000000,...,39.8,12.7,357.0,NaN,NaN,NaN,NaN,NaN,NaN,13.5
4,18208023,32018180,58,1,0,1,0,0,0,104.600000,...,NaN,NaN,NaN,3.7,NaN,NaN,NaN,149.0,51.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90620,15562868,38352081,69,0,0,1,0,0,0,100.000000,...,NaN,NaN,NaN,NaN,85.4,NaN,NaN,NaN,NaN,NaN
90621,11228961,39805704,77,1,0,0,0,0,1,80.000000,...,29.4,9.8,130.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0
90622,19805149,33741942,85,1,1,0,0,0,0,103.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90623,19065885,36148978,60,1,0,0,1,0,0,79.900000,...,26.1,8.5,97.0,NaN,NaN,NaN,NaN,NaN,NaN,24.9


In [4]:
def median_impute(df, columns):
    # df has column stay_id
    for col in columns:
        median = df[col].dropna(how='any').median() # global median
        stay_median = {}
        for stay,val in zip(df['stay_id'], df[col]):
            if stay not in stay_median:
                stay_median[stay] = []
            if not math.isnan(float(val)):
                stay_median[stay].append(float(val))
        stay_median1 = {}
        for i in stay_median:
            vals = stay_median[i]
            if len(vals) == 0:
                stay_median1[i] = median # global median
            else:
                stay_median1[i] = np.median(vals) # local median
        new_values = []
        for stay,val in zip(df['stay_id'], df[col]):
            if math.isnan(float(val)):
                new_values.append(stay_median1[stay])
            else:
                new_values.append(float(val))
        df[col] = new_values
    return df

def datetime_to_sec(df,time_col='charttime'):
    time_in_sec = []
    date_format = '%Y-%m-%d %H:%M:%S'
    for i in df[time_col]:
        time_in_sec.append(datetime.strptime(str(i),date_format).timestamp())
    df[time_col] = time_in_sec
    return df

def datetime_to_sec_entry(time):
    date_format = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(str(time),date_format).timestamp()

In [5]:
ppt_df = datetime_to_sec(ppt_df, time_col='pvl_charttime')
columns = ['aniongap', 'albumin','bicarbonate','creatinine','chloride', 'hematocrit',\
          'hemoglobin', 'platelet', 'potassium', 'ptt', 'inr', 'pt', 'sodium', 'bun', 'wbc'] # drop glucose
ppt_df = median_impute(ppt_df, columns)
ppt_df

,subject_id,stay_id,age,is_male,is_black,is_white,is_hispanic,is_asian,is_other,weight,...,hematocrit,hemoglobin,platelet,potassium,ptt,inr,pt,sodium,bun,wbc
0,16175244,36032605,32,1,0,1,0,0,0,82.200000,...,23.90,7.7,139.0,4.20,41.90,1.500,16.70,143.0,30.0,25.60
1,16133115,38070632,57,1,0,1,0,0,0,139.288889,...,25.40,8.3,152.0,4.20,56.15,1.400,15.00,136.0,50.5,9.80
2,18393915,32951543,65,0,0,1,0,0,0,60.200000,...,24.25,7.8,80.0,4.10,59.40,1.200,13.20,143.0,99.0,3.55
3,13060420,34563409,57,1,0,0,1,0,0,85.000000,...,39.80,12.7,357.0,3.90,38.45,1.175,12.50,145.0,29.5,13.50
4,18208023,32018180,58,1,0,1,0,0,0,104.600000,...,34.90,11.2,243.0,3.70,36.70,1.200,13.00,149.0,51.0,12.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90620,15562868,38352081,69,0,0,1,0,0,0,100.000000,...,31.25,9.5,288.0,4.30,85.40,1.400,15.70,135.0,24.0,23.90
90621,11228961,39805704,77,1,0,0,0,0,1,80.000000,...,29.40,9.8,130.0,4.10,61.00,1.300,13.90,142.0,68.0,15.00
90622,19805149,33741942,85,1,1,0,0,0,0,103.000000,...,35.90,11.6,234.0,4.85,32.50,1.700,19.00,143.0,79.0,11.80
90623,19065885,36148978,60,1,0,0,1,0,0,79.900000,...,26.10,8.5,97.0,3.80,44.00,1.100,12.15,136.5,27.5,24.90


### Extract vital signs and impute

In [6]:
def feature_dict(df, feature_start=2, stay=0, time=1):
    result = {}
    n = len(df)
    for i in range(n):
        row = list(df.iloc[i])
        stay_id = row[stay]
        charttime = row[time]
        features = row[feature_start:]
        if stay_id not in result:
            result[stay_id] = {}
        result[stay_id][charttime] = features
    return result 

file = './vital_df.pickle'
if os.path.exists(file):
    file = open(file,'rb')
    vital_dict = pickle.load(file) # {'stay_id:{charttime1:[features], charttime2:[features]}'}
else:    
    stay_id = []
    for i in ppt_df['stay_id']:
        stay_id.append(int(i))


    query_vital = '''
    select stay_id, charttime, heart_rate, sbp,dbp,mbp,resp_rate,temperature,spo2,glucose 
    from public.vitalsign where stay_id in (select distinct stay_id from public.chartevents where itemid = 229375)
    ''' 
    vital_df = pd.read_sql_query(query_vital,conn) 
    exclude = []
    row = 0
    for i in vital_df['stay_id']:
        if int(i) not in stay_id:
            exclude.append(row)
        row += 1
    vital_df.drop(exclude) # [stay_id, charttime, 'heart_rate', 'sbp','dbp','mbp','resp_rate','temperature','spo2','glucose']
    columns = ['heart_rate', 'sbp','dbp','mbp','resp_rate','temperature','spo2','glucose']
    vital_df = median_impute(vital_df,columns)
    vital_df = datetime_to_sec(vital_df)
    vital_dict = vital_df = feature_dict(vital_df, feature_start=2) # {'stay_id:{charttime1:[features], charttime2:[features]}'}
    file = open(file,"wb")
    pickle.dump(vital_dict, file)

### Convert lab DF into dictionary

In [7]:
stay_start = {}
for i,j in zip(ppt_df['stay_id'], ppt_df['pvl_charttime']):
    if i not in stay_start:
        stay_start[i] = []
    stay_start[i].append(j)
for i in stay_start:
    stay_start[i] = min(stay_start[i])

columns = ['subject_id','stay_id', 'pvl_charttime','age', 'is_male', 'is_black',  'is_white',\
            'is_asian','is_hispanic', 'is_other',  'weight', \
           'aniongap', 'albumin','bicarbonate','creatinine',\
           'chloride', 'hematocrit','hemoglobin', 'platelet', 'potassium', 'ptt', 'inr', 'pt', \
           'sodium', 'bun', 'wbc']
ppt_dict = ppt_df = feature_dict(ppt_df[columns], feature_start=3, stay=1, time=2) # [subject_id, stay_id, hadm_id,...]
# {'stay_id:{charttime1:[features], charttime2:[features]}'}

## Merge lab and vital signs

In [8]:
structural_features= {} # {'stay_id:{charttime1:[features], charttime2:[features]}'}
for stay in stay_start:
    start = stay_start[stay]
    lab_times = sorted(ppt_dict[stay].keys())
    temp = {}
    for vital_time in vital_dict[stay]:
        idx = bisect.bisect_left(lab_times, vital_time) - 1
        if idx < 0:
            continue
        lab_time = lab_times[idx]
        merged_features = copy.deepcopy(ppt_dict[stay][lab_time])
        merged_features.extend(vital_dict[stay][vital_time])
        temp[vital_time] = merged_features
    if len(temp) == 0:
        continue
    else:
        structural_features[stay] = temp

#del ppt_dict
#del vital_dict

## Extract treatment

In [1]:
file = './treatment.pickle'
if os.path.exists(file): #os.path.exists(file)
    file = open(file,'rb')
    treatment_dict = pickle.load(file) #  {stay_id:{time1:dosage1, time2:dosage2}}
else:
    treatment_query = '''
    select stay_id, charttime, valuenum as level 
    from public.chartevents where itemid=229375
    '''
    treatment_df = pd.read_sql_query(treatment_query,conn)
    treatment_dict = {}
    n = len(treatment_df)
    for i in range(n):
        stay_id, time, amount = list(treatment_df[['stay_id', 'charttime', 'level']].iloc[i])
        if stay_id not in stay_start:
            continue
        else:
            if stay_id not in treatment_dict:
                treatment_dict[stay_id] = {}
            treatment_dict[stay_id][datetime_to_sec_entry(time)] = amount
    del treatment_df
    file = open(file,"wb")
    pickle.dump(treatment_dict, file)

NameError: name 'os' is not defined

## Extract outcome

In [10]:
file = './outocmes.pickle'
if os.path.exists(file): #os.path.exists(file)
    file = open(file,'rb')
    outcome_dict = pickle.load(file) #  {stay_id:{time1:SAS1, time2:SAS2}}
else:
    outcome_query = '''
    select icu.stay_id, lab.charttime, lab.valuenum from public.labevents lab join public.icustays icu on 
    lab.hadm_id = icu.hadm_id where lab.itemid = 51275 and icu.intime <= lab.charttime and icu.outtime >= lab.charttime
    '''
    outcome_df = pd.read_sql_query(outcome_query,conn)
    outcome_dict = {}
    n = len(outcome_df)
    for i in range(n):
        stay_id, time, ss = list(outcome_df[['stay_id', 'charttime', 'valuenum']].iloc[i])
        if stay_id not in stay_start:
            continue
        else:
            if stay_id not in outcome_dict:
                outcome_dict[stay_id] = {}
            outcome_dict[stay_id][datetime_to_sec_entry(time)] = ss
    file = open(file,"wb")
    pickle.dump(outcome_dict, file)
    del outcome_df
    

## Merge features, dosage, outcome

In [11]:
## structural_features= {}  {'stay_id:{charttime1:[features], charttime2:[features]}'}
## treatment_dict = {} {stay_id:{time1:dosage1, time2:dosage2}}
## outcome_dict = {} {stay_id:{time1:outcome1, time2:outcome2}}
## stay_start {stay_id: starttime}

unique_stay = set()
all_data = []
maxtime_after_treatment = 20*60 # sec, the maximum time after treatment that the label should be recorded

for stay in treatment_dict:
    if stay not in structural_features or stay not in outcome_dict:
        continue
    for treat_start in treatment_dict[stay]:
        temp = []
        # find features before treatment
        feature_times = sorted(list(structural_features[stay]))
        idx = bisect.bisect_left(feature_times, treat_start) - 1
        if idx < 0:
            continue
        features = structural_features[stay][feature_times[idx]]
        # find dosage
        dosage = treatment_dict[stay][treat_start]
        
        label_times = sorted(list(outcome_dict[stay]))
        idx = bisect.bisect_left(label_times, treat_start) - 1
        if idx < 0: 
            continue
        
        try:
            label_time = label_times[idx + 1]
        except:
            continue # no label
        # check if label is recorded too late
        if label_time - treat_start > maxtime_after_treatment:
            continue
        label_after_t = outcome_dict[stay][label_time]
        temp.append(label_after_t)
        temp.append(dosage)
        temp.extend(features)
        all_data.append(temp)
        unique_stay.add(stay)

### Store data

In [14]:
for i in [structural_features, treatment_dict, outcome_dict, stay_start]:
    del i
def write_csv(outpath, data):
    with open(outpath, 'w',newline='') as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(data)
file = './mimiciv_coag_raw.csv'
write_csv(file, all_data)

### Find unique patient

In [19]:
sepsis_query = '''
select icu.subject_id, ss.stay_id from public.sepsis3 ss join public.icustays icu on 
ss.stay_id=icu.stay_id
'''
sepsis_df = pd.read_sql_query(sepsis_query,conn)
sepsis_dict = {} # {stay_id:subject_id}
n = len(sepsis_df)
for i in range(n):
    subject, stay = list(sepsis_df[['subject_id', 'stay_id']].iloc[i])
    if stay not in sepsis_dict:
        sepsis_dict[stay] = []
    sepsis_dict[stay].append(subject)
    
unique_subject = set()
for i in unique_stay:
    for j in sepsis_dict[i]:
        unique_subject.add(j)
print(('The number of unique subject: ', len(unique_subject)))

('The number of unique subject: ', 389)
